In [ ]:
# Import Library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import os
import time
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix, f1_score

# Impor modul lokal (karena notebook ada di folder lstm/)
import config
from data_preprocessing import load_data, create_text_vectorizer, compute_f1_score
from model import create_lstm_model # Spesifik untuk LSTM
from train import train_and_evaluate_lstm_model, plot_training_history # Spesifik untuk LSTM

# Atur seed untuk reproduksibilitas
np.random.seed(config.RANDOM_SEED)
tf.random.set_seed(config.RANDOM_SEED)

# Opsi tampilan Pandas (opsional, untuk tabel yang lebih baik)
pd.set_option('display.max_colwidth', 100) 
pd.set_option('display.width', 1000)

In [ ]:
# Muat data
(train_texts, train_labels), (valid_texts, valid_labels), (test_texts, test_labels), label_mapping, num_classes = load_data()

# Tampilkan informasi data
print(f"Dataset sizes: Train={len(train_texts)}, Valid={len(valid_texts)}, Test={len(test_texts)}")
print(f"Kelas dalam dataset: {list(label_mapping.keys())} -> {label_mapping}")
print(f"Jumlah kelas: {num_classes}")
print(f"Distribusi label train: {np.bincount(train_labels)}")
print(f"Distribusi label valid: {np.bincount(valid_labels)}")
print(f"Distribusi label test: {np.bincount(test_labels)}")

# Visualisasi distribusi kelas pada data training
plt.figure(figsize=(8, 5))
class_counts_train = np.bincount(train_labels)
plt.bar(list(label_mapping.keys()), class_counts_train, color=['red', 'lightgrey', 'green'])
plt.title('Distribusi Kelas di Data Training (LSTM)')
plt.xlabel('Label')
plt.ylabel('Jumlah Sampel')
plt.grid(axis='y', linestyle='--')
plt.show()

# Buat text vectorizer menggunakan data training
# Parameter diambil dari config.py
vectorizer, vocab, vocab_size = create_text_vectorizer(
    train_texts, 
    max_tokens=config.MAX_TOKENS, 
    output_sequence_length=config.OUTPUT_SEQ_LEN
)
print(f"\nUkuran Vocabulary (dari TextVectorization): {vocab_size}")
print(f"Contoh beberapa token dari vocabulary: {vocab[100:110]}")

# Tampilkan beberapa contoh teks dan label yang sudah di-map ke integer
print("\nContoh data training:")
for i in range(3):
    sentiment = list(label_mapping.keys())[train_labels[i]]
    print(f"Teks: {train_texts[i][:100]}... | Label Asli: {sentiment} | Label Numerik: {train_labels[i]}")
    vectorized_text = vectorizer([train_texts[i]])
    print(f"Teks Vektorisasi (awal): {vectorized_text.numpy()[0, :15]}...")

# Visualisasikan panjang teks (jumlah kata)
text_lengths_train = [len(text.split()) for text in train_texts]
plt.figure(figsize=(10, 5))
plt.hist(text_lengths_train, bins=50, color='skyblue', edgecolor='black')
plt.title('Distribusi Panjang Teks di Data Training (jumlah kata) - LSTM')
plt.xlabel('Jumlah Kata')
plt.ylabel('Frekuensi')
plt.axvline(np.mean(text_lengths_train), color='red', linestyle='dashed', linewidth=1, label=f'Mean: {np.mean(text_lengths_train):.1f}')
plt.axvline(np.median(text_lengths_train), color='green', linestyle='dashed', linewidth=1, label=f'Median: {np.median(text_lengths_train):.1f}')
plt.axvline(config.OUTPUT_SEQ_LEN, color='purple', linestyle='dotted', linewidth=2, label=f'Output Seq Len: {config.OUTPUT_SEQ_LEN}')
plt.legend()
plt.grid(True, linestyle=':', alpha=0.7)
plt.show()

print(f"Panjang rata-rata (kata): {np.mean(text_lengths_train):.1f}")
print(f"Panjang median (kata): {np.median(text_lengths_train):.1f}")
print(f"Panjang maksimum (kata): {np.max(text_lengths_train)}")
print(f"Panjang teks yang dipilih untuk sekuens (OUTPUT_SEQ_LEN): {config.OUTPUT_SEQ_LEN}")

In [ ]:
# Cek parameter default untuk LSTM dari config.py
print("Parameter model LSTM default dari config.py:")
print(f"- Embedding dimension: {config.EMBEDDING_DIM}")
print(f"- LSTM units: {config.LSTM_UNITS}")
print(f"- Number of LSTM layers: {config.NUM_LSTM_LAYERS}")
print(f"- Bidirectional LSTM: {config.BIDIRECTIONAL_LSTM}")
print(f"- Dropout rate (setelah LSTM/Dense): {config.DROPOUT_RATE}")
print(f"- Embedding Dropout rate (SpatialDropout1D): {config.EMBEDDING_DROPOUT}")
print(f"- Recurrent Dropout rate (dalam LSTM cell): {config.RECURRENT_DROPOUT_LSTM}")
print(f"- L2 Regularization: {config.L2_REG}")
print(f"- Learning Rate: {config.LEARNING_RATE}")

print(f"\nParameter training dari config.py:")
print(f"- Batch size: {config.BATCH_SIZE}")
print(f"- Max epochs: {config.EPOCHS}")
print(f"- Early Stopping Patience: {config.ES_PATIENCE}")
print(f"- ReduceLROnPlateau Factor: {config.LR_FACTOR}")
print(f"- ReduceLROnPlateau Patience: {config.LR_PATIENCE}")
print(f"- Minimum Learning Rate: {config.MIN_LR}")

# Parameter data preprocessing dari config.py
print(f"\nParameter data preprocessing dari config.py:")
print(f"- Max tokens (vocabulary size): {config.MAX_TOKENS}")
print(f"- Output sequence length: {config.OUTPUT_SEQ_LEN}")

In [ ]:
# Buat folder untuk menyimpan hasil jika belum ada
# (train_and_evaluate_lstm_model juga akan membuat ini, tapi bisa juga di sini)
os.makedirs("models", exist_ok=True)
os.makedirs("checkpoints", exist_ok=True)
# Untuk plot dari notebook jika berbeda dari train.py
os.makedirs("plots_notebook_lstm", exist_ok=True) 

# Latih model LSTM baseline menggunakan parameter default dari config.py
print("\nMemulai pelatihan model LSTM baseline...")
model_baseline_lstm, history_baseline_lstm, preds_baseline_lstm, labels_true_baseline_lstm, vectorizer_baseline_lstm, metrics_baseline_lstm = train_and_evaluate_lstm_model(
    model_name="baseline_lstm_model" # Nama model bisa disesuaikan
    # Parameter lain akan menggunakan default dari config.py yang di-pass di dalam fungsi
)

# Plot kurva training untuk model baseline
# Jika Anda ingin menyimpan plot dari notebook ke folder lain:
# plot_training_history(history_baseline_lstm, "baseline_lstm_model", save_dir="plots_notebook_lstm")
plot_training_history(history_baseline_lstm, "baseline_lstm_model") # Menggunakan fungsi dari train.py

# Tampilkan metrik evaluasi
print("\nMetrik evaluasi model LSTM baseline:")
for name, value in metrics_baseline_lstm.items():
    if isinstance(value, float):
        print(f"- {name}: {value:.4f}")
    else:
        print(f"- {name}: {value}")


# Visualisasi Confusion Matrix untuk model baseline
pred_classes_baseline_lstm = np.argmax(preds_baseline_lstm, axis=1)
cm_baseline_lstm = confusion_matrix(labels_true_baseline_lstm, pred_classes_baseline_lstm)

plt.figure(figsize=(8, 6))
plt.imshow(cm_baseline_lstm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix - Model LSTM Baseline')
plt.colorbar(label='Jumlah Sampel')

tick_marks = np.arange(len(list(label_mapping.keys())))
plt.xticks(tick_marks, list(label_mapping.keys()), rotation=45, ha="right")
plt.yticks(tick_marks, list(label_mapping.keys()))

# Tambahkan teks nilai ke confusion matrix
thresh = cm_baseline_lstm.max() / 2.
for i in range(cm_baseline_lstm.shape[0]):
    for j in range(cm_baseline_lstm.shape[1]):
        plt.text(j, i, format(cm_baseline_lstm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm_baseline_lstm[i, j] > thresh else "black")

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
# plt.savefig("plots_notebook_lstm/cm_baseline_lstm.png") # Jika ingin menyimpan
plt.show()

# Tampilkan laporan klasifikasi lengkap
print("\nLaporan Klasifikasi Model LSTM Baseline:")
print(classification_report(labels_true_baseline_lstm, pred_classes_baseline_lstm, target_names=list(label_mapping.keys()), zero_division=0))

In [ ]:
# Direktori untuk menyimpan hasil eksperimen LSTM
RESULTS_DIR_LSTM = "experiment_results_lstm"
os.makedirs(RESULTS_DIR_LSTM, exist_ok=True)
print(f"Hasil eksperimen LSTM akan disimpan di: {RESULTS_DIR_LSTM}")

# Definisikan variasi parameter yang akan diuji untuk LSTM
# Sesuai spesifikasi: 3 variasi jumlah layer, 3 variasi unit, 2 variasi arah
LSTM_LAYERS_VARIATIONS = [1, 2, 3]  # Jumlah layer LSTM
LSTM_UNITS_VARIATIONS = [32, 64, 128] # Jumlah unit per layer LSTM (sesuaikan jika perlu)
BIDIRECTIONAL_LSTM_VARIATIONS = [False, True]  # Unidirectional vs Bidirectional LSTM

In [ ]:
def run_lstm_layer_count_experiment():
    """Jalankan eksperimen untuk menguji pengaruh jumlah layer LSTM"""
    print("\n" + "="*60)
    print("EKSPERIMEN LSTM: VARIASI JUMLAH LAYER LSTM")
    print("="*60)
    
    results = []
    histories = {}
    
    # Uji untuk setiap jumlah layer LSTM
    for num_layers in LSTM_LAYERS_VARIATIONS:
        model_name = f"lstm_layers_{num_layers}"
        print(f"\n--- Menguji model LSTM dengan {num_layers} layer ---")
        
        # Latih model dengan variasi ini
        # Parameter lain menggunakan default dari config.py (via train_and_evaluate_lstm_model)
        _, history, _, _, _, metrics = train_and_evaluate_lstm_model(
            num_lstm_layers=num_layers, # Override jumlah layer LSTM
            model_name=model_name
        )
        
        # Simpan hasilnya
        results.append({
            'num_lstm_layers': num_layers,
            'accuracy': metrics['test_accuracy'],
            'f1_score': metrics['test_f1'],
            'loss': metrics['test_loss'],
            'train_time': metrics['training_time']
        })
        
        # Simpan history untuk plot
        histories[num_layers] = history
    
    # Konversi ke DataFrame untuk analisis lebih mudah
    results_df = pd.DataFrame(results)
    
    # Simpan hasil ke CSV
    csv_path = f"{RESULTS_DIR_LSTM}/lstm_layer_count_results.csv"
    results_df.to_csv(csv_path, index=False)
    print(f"\nHasil eksperimen jumlah layer LSTM disimpan di {csv_path}")
    
    return results_df, histories

# Jalankan eksperimen variasi jumlah layer LSTM
layer_results_df_lstm, layer_histories_lstm = run_lstm_layer_count_experiment()

In [ ]:
# Tampilkan hasil dalam tabel
print("\nHasil Eksperimen Jumlah Layer LSTM:")
display(layer_results_df_lstm)

# Visualisasikan pengaruh jumlah layer LSTM terhadap performa
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(layer_results_df_lstm['num_lstm_layers'], layer_results_df_lstm['accuracy'], 'o-', label='Accuracy')
plt.plot(layer_results_df_lstm['num_lstm_layers'], layer_results_df_lstm['f1_score'], 's--', label='F1 Score (Macro)')
plt.xlabel('Jumlah Layer LSTM')
plt.ylabel('Skor')
plt.title('Pengaruh Jumlah Layer LSTM terhadap Performa')
plt.xticks(layer_results_df_lstm['num_lstm_layers']) # Pastikan semua nilai variasi ditampilkan
plt.grid(True, linestyle=':', alpha=0.7)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(layer_results_df_lstm['num_lstm_layers'], layer_results_df_lstm['train_time'], 'o-', color='green', label='Waktu Training')
plt.xlabel('Jumlah Layer LSTM')
plt.ylabel('Waktu Training (detik)')
plt.title('Pengaruh Jumlah Layer LSTM terhadap Waktu Training')
plt.xticks(layer_results_df_lstm['num_lstm_layers'])
plt.grid(True, linestyle=':', alpha=0.7)
plt.legend()

plt.tight_layout()
# plt.savefig(f"{RESULTS_DIR_LSTM}/plot_lstm_layer_performance.png")
plt.show()

# Plot kurva learning untuk setiap variasi jumlah layer LSTM
num_variations = len(LSTM_LAYERS_VARIATIONS)
plt.figure(figsize=(15, 5 * num_variations)) # Lebar, tinggi per subplot

for i, layers in enumerate(LSTM_LAYERS_VARIATIONS):
    history_data = layer_histories_lstm[layers].history
    
    # Plot Loss
    plt.subplot(num_variations, 2, 2*i + 1)
    plt.plot(history_data['loss'], label=f'Training Loss ({layers} layers)')
    plt.plot(history_data['val_loss'], '--', label=f'Validation Loss ({layers} layers)')
    plt.title(f'Kurva Loss - Model LSTM dengan {layers} Layer')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.7)
    
    # Plot Accuracy
    plt.subplot(num_variations, 2, 2*i + 2)
    plt.plot(history_data['accuracy'], label=f'Training Accuracy ({layers} layers)')
    plt.plot(history_data['val_accuracy'], '--', label=f'Validation Accuracy ({layers} layers)')
    plt.title(f'Kurva Akurasi - Model LSTM dengan {layers} Layer')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.7)

plt.tight_layout()
# plt.savefig(f"{RESULTS_DIR_LSTM}/plot_lstm_layer_learning_curves.png")
plt.show()

In [ ]:
def run_lstm_unit_count_experiment():
    """Jalankan eksperimen untuk menguji pengaruh jumlah unit LSTM per layer"""
    print("\n" + "="*60)
    print("EKSPERIMEN LSTM: VARIASI JUMLAH UNIT LSTM PER LAYER")
    print("="*60)
    
    results = []
    histories = {}
    
    # Uji untuk setiap jumlah unit LSTM
    for num_units in LSTM_UNITS_VARIATIONS:
        model_name = f"lstm_units_{num_units}"
        print(f"\n--- Menguji model LSTM dengan {num_units} unit per layer ---")
        
        # Latih model dengan variasi ini
        # Parameter lain menggunakan default dari config.py
        _, history, _, _, _, metrics = train_and_evaluate_lstm_model(
            lstm_units=num_units, # Override jumlah unit LSTM
            model_name=model_name
        )
        
        # Simpan hasilnya
        results.append({
            'num_lstm_units': num_units,
            'accuracy': metrics['test_accuracy'],
            'f1_score': metrics['test_f1'],
            'loss': metrics['test_loss'],
            'train_time': metrics['training_time']
        })
        
        # Simpan history untuk plot
        histories[num_units] = history
        
    results_df = pd.DataFrame(results)
    csv_path = f"{RESULTS_DIR_LSTM}/lstm_unit_count_results.csv"
    results_df.to_csv(csv_path, index=False)
    print(f"\nHasil eksperimen jumlah unit LSTM disimpan di {csv_path}")
    
    return results_df, histories

# Jalankan eksperimen variasi jumlah unit LSTM
unit_results_df_lstm, unit_histories_lstm = run_lstm_unit_count_experiment()

In [ ]:
# Tampilkan hasil dalam tabel
print("\nHasil Eksperimen Jumlah Unit LSTM:")
display(unit_results_df_lstm)

# Visualisasikan pengaruh jumlah unit LSTM
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
plt.plot(unit_results_df_lstm['num_lstm_units'], unit_results_df_lstm['accuracy'], 'o-', label='Accuracy')
plt.plot(unit_results_df_lstm['num_lstm_units'], unit_results_df_lstm['f1_score'], 's--', label='F1 Score (Macro)')
plt.xlabel('Jumlah Unit LSTM per Layer')
plt.ylabel('Skor')
plt.title('Pengaruh Jumlah Unit LSTM terhadap Performa')
plt.xticks(unit_results_df_lstm['num_lstm_units'])
plt.grid(True, linestyle=':', alpha=0.7)
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(unit_results_df_lstm['num_lstm_units'], unit_results_df_lstm['train_time'], 'o-', color='green', label='Waktu Training')
plt.xlabel('Jumlah Unit LSTM per Layer')
plt.ylabel('Waktu Training (detik)')
plt.title('Pengaruh Jumlah Unit LSTM terhadap Waktu Training')
plt.xticks(unit_results_df_lstm['num_lstm_units'])
plt.grid(True, linestyle=':', alpha=0.7)
plt.legend()

plt.tight_layout()
# plt.savefig(f"{RESULTS_DIR_LSTM}/plot_lstm_unit_performance.png")
plt.show()

# Plot kurva learning untuk setiap variasi jumlah unit LSTM
num_variations = len(LSTM_UNITS_VARIATIONS)
plt.figure(figsize=(15, 5 * num_variations))

for i, units in enumerate(LSTM_UNITS_VARIATIONS):
    history_data = unit_histories_lstm[units].history
    
    plt.subplot(num_variations, 2, 2*i + 1)
    plt.plot(history_data['loss'], label=f'Training Loss ({units} units)')
    plt.plot(history_data['val_loss'], '--', label=f'Validation Loss ({units} units)')
    plt.title(f'Kurva Loss - Model LSTM dengan {units} Unit')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.7)
    
    plt.subplot(num_variations, 2, 2*i + 2)
    plt.plot(history_data['accuracy'], label=f'Training Accuracy ({units} units)')
    plt.plot(history_data['val_accuracy'], '--', label=f'Validation Accuracy ({units} units)')
    plt.title(f'Kurva Akurasi - Model LSTM dengan {units} Unit')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.7)

plt.tight_layout()
# plt.savefig(f"{RESULTS_DIR_LSTM}/plot_lstm_unit_learning_curves.png")
plt.show()

In [ ]:
def run_lstm_bidirectional_experiment():
    """Jalankan eksperimen untuk membandingkan LSTM satu arah vs dua arah"""
    print("\n" + "="*60)
    print("EKSPERIMEN LSTM: VARIASI ARAH LAYER LSTM (BIDIRECTIONAL)")
    print("="*60)
    
    results = []
    histories = {}
    
    # Uji untuk setiap setting bidirectional
    for is_bidirectional in BIDIRECTIONAL_LSTM_VARIATIONS:
        direction_name = "bidirectional_lstm" if is_bidirectional else "unidirectional_lstm"
        model_name = f"lstm_{direction_name}"
        print(f"\n--- Menguji model dengan LSTM {direction_name} ---")
        
        # Latih model dengan variasi ini
        _, history, _, _, _, metrics = train_and_evaluate_lstm_model(
            bidirectional=is_bidirectional, # Override parameter bidirectional
            model_name=model_name
        )
        
        # Simpan hasilnya
        results.append({
            'is_bidirectional_lstm': is_bidirectional,
            'direction_lstm': direction_name,
            'accuracy': metrics['test_accuracy'],
            'f1_score': metrics['test_f1'],
            'loss': metrics['test_loss'],
            'train_time': metrics['training_time']
        })
        
        # Simpan history untuk plot
        histories[direction_name] = history
        
    results_df = pd.DataFrame(results)
    csv_path = f"{RESULTS_DIR_LSTM}/lstm_bidirectional_results.csv"
    results_df.to_csv(csv_path, index=False)
    print(f"\nHasil eksperimen arah LSTM disimpan di {csv_path}")
    
    return results_df, histories

# Jalankan eksperimen variasi arah LSTM
bidirectional_results_df_lstm, bidirectional_histories_lstm = run_lstm_bidirectional_experiment()

In [ ]:
# Tampilkan hasil dalam tabel
print("\nHasil Eksperimen Arah Layer LSTM:")
display(bidirectional_results_df_lstm)

# Visualisasikan pengaruh arah LSTM dengan chart batang
plt.figure(figsize=(10, 7)) # Sedikit lebih tinggi untuk label
directions_lstm = bidirectional_results_df_lstm['direction_lstm'].tolist()
x_pos = np.arange(len(directions_lstm))
bar_width = 0.35

fig, ax = plt.subplots(figsize=(10, 6)) # Buat figure dan axes terpisah
rects1 = ax.bar(x_pos - bar_width/2, bidirectional_results_df_lstm['accuracy'], bar_width, label='Accuracy', color='skyblue')
rects2 = ax.bar(x_pos + bar_width/2, bidirectional_results_df_lstm['f1_score'], bar_width, label='F1 Score (Macro)', color='lightcoral')

ax.set_title('Perbandingan Performa LSTM Unidirectional vs Bidirectional')
ax.set_ylabel('Skor')
ax.set_xticks(x_pos)
ax.set_xticklabels(directions_lstm, rotation=15, ha="right") # Rotasi label jika panjang
ax.legend(loc='upper left', bbox_to_anchor=(1,1)) # Pindahkan legend jika perlu
ax.grid(axis='y', linestyle=':', alpha=0.7)

# Fungsi untuk menambahkan label nilai di atas bar
def autolabel_bars(rects, ax_obj):
    for rect in rects:
        height = rect.get_height()
        ax_obj.annotate(f'{height:.3f}',
                       xy=(rect.get_x() + rect.get_width() / 2, height),
                       xytext=(0, 3), # 3 points vertical offset
                       textcoords="offset points",
                       ha='center', va='bottom', fontsize=9)

autolabel_bars(rects1, ax)
autolabel_bars(rects2, ax)

plt.tight_layout(rect=[0, 0, 0.85, 1]) # Sesuaikan layout untuk legend di luar
# plt.savefig(f"{RESULTS_DIR_LSTM}/plot_lstm_bidirectional_performance.png")
plt.show()


# Plot kurva learning untuk perbandingan bidirectional vs unidirectional LSTM
num_variations = len(BIDIRECTIONAL_LSTM_VARIATIONS)
plt.figure(figsize=(15, 5 * num_variations))

for i, (direction, history_obj) in enumerate(bidirectional_histories_lstm.items()):
    history_data = history_obj.history
    
    plt.subplot(num_variations, 2, 2*i + 1)
    plt.plot(history_data['loss'], label=f'Training Loss ({direction})')
    plt.plot(history_data['val_loss'], '--', label=f'Validation Loss ({direction})')
    plt.title(f'Kurva Loss - Model {direction}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.7)
    
    plt.subplot(num_variations, 2, 2*i + 2)
    plt.plot(history_data['accuracy'], label=f'Training Accuracy ({direction})')
    plt.plot(history_data['val_accuracy'], '--', label=f'Validation Accuracy ({direction})')
    plt.title(f'Kurva Akurasi - Model {direction}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True, linestyle=':', alpha=0.7)

plt.tight_layout()
# plt.savefig(f"{RESULTS_DIR_LSTM}/plot_lstm_bidirectional_learning_curves.png")
plt.show()